In [1]:
from transformer_nb2 import *
from dataset import make_data_generator
import json
from tqdm import tqdm_notebook as tqdm
#from torchsummary import summary

In [2]:
folder = 'data/food/'
data_name = folder+'data.json'
# validation_name = folder+'valid_seq.json'
# testdata_name = folder+'testdata_seq.json'
vocab_name = folder+'vocab.json'

In [3]:
num_epochs = 10
save_rate = 1 #how many epochs per modelsave
#continue_from = "trained/Model1" # if none, put None
continue_from = None
epsilon = 1e-8
validation_size = 10000
device = torch.device('cuda')
!mkdir -p trained

In [4]:
vocab = json.load(open(vocab_name, 'r'))
VOC_SIZE = len(vocab)
INPUT_MAX = 100
SUMM_MAX = 100
UNK = "[UNK]"
BOS = "[CLS]"
EOS = "[SEP]"
PAD = "[PAD]"

In [5]:
batch_size = 64

training_set, training_generator = make_data_generator(\
data_name, INPUT_MAX, SUMM_MAX, vocab[PAD], batch_size, pretrain=True, cutoff=None, shuffle=True, num_workers=4)

# validation_set, validation_generator = make_data_generator(\
# validation_name, INPUT_MAX, OUTPUT_MAX, vocab[PAD], batch_size, cutoff=validation_size, shuffle=False, num_workers=4)

def data_gen_train():
    for src, tgt in training_generator:
        src = src.to(device)
        tgt = tgt.to(device)
        b = Batch(src, tgt, vocab[PAD])
        yield b

loading json
load json done.



(tensor([ 4149,  1997,  3504,  1997,  3012,  1998,  3899,  2084,  2026,  2062,
         6799,  2009,  1045,  2031, 13995,  8995,  2022,  1037,  1037,  2023,
         2833,  2003,  2084,  2100, 20717,  2195,  1012,  4031,   102,  2066,
         1996,  1998,  2000,  1996,  2035,  2204, 14747,  2488,  2087,  9120,
        10346,  2016, 27141,  4031,  3688,  1012,  2179,  1998,   101,  6240,
         2068,  2015,  1012, 18604,  2031,  3737,  2488,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0]), tensor([  101,  1045,  2031,  4149,  2195,  1997,  1996,  8995,  3012, 27141,
         3899,  2833,  3688,  1998,  2031,  2179,  2068,  2035,  2000,  2022,
         1997,  2204,  3737,  1012,  1996,  4031,  3504,  20

In [6]:
import math
total_train = int(math.ceil(training_set.size / batch_size))
# total_valid = int(math.ceil(validation_set.size / batch_size))
# print(total_train, total_valid)

In [7]:
def init_param(model):
    for p in model.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)

# translator = PointerGenerator(
#             hidden_dim=256, 
#             emb_dim=256, 
#             input_len=INPUT_MAX, 
#             output_len=SUMM_MAX, 
#             voc_size=VOC_SIZE, 
#             eps=1e-8
#         ).to(device)
translator = make_model(VOC_SIZE, VOC_SIZE, N=4, d_model=128, d_ff=256, h=4, dropout=0.1, emb_share=True).to(device)

criterion = LabelSmoothing(size=VOC_SIZE, padding_idx=vocab[PAD], smoothing=0.1).to(device)
model_opt = torch.optim.Adam(translator.parameters(), lr=1e-4, betas=(0.9, 0.998), eps=1e-8)
loss_compute = SimpleLossCompute(translator.generator, criterion, model_opt)

/home/marvin/anaconda3/envs/py37cuda10/lib/python3.7/site-packages/torch/nn/_reduction.py:49: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


In [8]:
# import torch.nn.functional as F
# def loss_compute(x, y, norm): 
#     x = F.log_softmax(x)
    
#     print(x.shape)
    
#     loss = criterion(x.contiguous().view(-1, x.size(-1)), 
#                           y.contiguous().view(-1)) / norm
#     loss.backward()
#     if model_opt is not None:
#         model_opt.step()
#         model_opt.zero_grad()
#     return loss.item() * norm

In [ ]:
# model = make_big_bird(vocab, N=4, d_model=256, d_ff=512, h=8, dropout=0.1, emb_share=True, bert_share=True)
#model.load("Nest/NewbornBird")

In [ ]:
vocab_inv = {a:b for b, a in vocab.items()}
def convert_ids_to_tokens(ids):
    return [vocab_inv[i] for i in ids]

In [ ]:
start = 1 if continue_from == None else (int(continue_from.split("Model")[-1])+1)
history = []


for epoch in range(start, num_epochs+1):
    print("Epoch", epoch)
    
    # training
    stats = Stats()
    translator.train()
    trange = tqdm(enumerate(data_gen_train()), total=total_train)
    
    for i, batch in trange:
        out = translator.forward(batch.src, batch.trg, batch.src_mask, batch.trg_mask)
        loss = loss_compute(out, batch.trg_y, batch.ntokens)
        stats.update(loss, batch.ntokens, log=1)
        
        if( i % 150 == 0):
            probs = translator.generator(out) 
            print("\n")
            next_words = torch.argmax(probs, dim=-1, keepdim=True)            
            print(convert_ids_to_tokens([i.item() for i in next_words[0]]))
            print(convert_ids_to_tokens([i.item() for i in batch.trg_y[0]]))
        
        trange.set_postfix(
            **{'loss': '{:.3f}'.format(loss)}
        )
        stats.update(loss, 1, log=0)
        
    t_h = stats.history
    history.append(t_h)
    
    print("[info] epoch train loss:", np.mean(t_h))
    
    try:
        !mkdir -p pretrained
        torch.save({'model':translator.state_dict(), 'training_history':t_h}, 
                   "pretrained/Translator"+str(epoch))
    except:
        continue

Epoch 1


Step: 1 Loss: 8.957157 Tokens / Sec: 8958.600531

['rector', 'greatest', '##oin', 'list', 'dramas', 'induced', 'newsletter', '##ype', 'cared', '##ype', 'greatest', 'undo', 'greatest', 'cared', 'bra', 'cared', '##np', 'newsletter', 'elector', '##np', 'induced', 'newsletter', 'dramas', '747', 'valid', 'greatest', 'cared', 'greatest', 'cared', 'greatest', 'dramas', 'feeding', '##lford', 'greatest', 'depression', 'cared', 'cared', 'pba', 'offs', 'crew', 'valid', 'cared', 'feeding', 'feeding', 'list', 'art', 'greatest', 'baba', 'greatest', 'agreement', 'dramas', 'greatest', 'newsletter', 'baba', '##np', 'depression', 'greatest', 'greatest', 'greatest', 'elector', 'pba', 'pba', 'agreement', 'depression', '##plicity', 'crew', '##wan', 'pba', '##np', 'art', 'art', 'elector', 'pba', '##wan', 'depression', '##wan', 'greatest', 'depression', '##wan', '##wan', '##wan', '##wan', 'dramas', 'newborn', 'managing', 'agreement', '##wan', '##wan', 'pba', 'feeding', '##wan', 'newsletter', 'managing', 'fee

In [ ]:
history = np.asarray(history).flatten()